## Alocação de unidades de atendimento móvel do CBMERJ de Campos dos Goytacazes - RJ

In [1]:
# Library
import pathlib
import os
import sys

from dotenv import load_dotenv
import numpy as np
import pandas as pd
from pulp import *
from utils.process import stander_text
# my packges
sys.path.insert(0, str(pathlib.Path().absolute().parent))

import process.allocation as allocation
from process.geocode import *

load_dotenv()


True

**Importar dados** com os endereços dos clientes(bairros) e facilidades(bases)

In [2]:
# Bairros (clientes)
df_bairro = pd.read_csv('../data/bairros.csv')
df_bairro['Endereco'] = df_bairro['Bairros'] + ', Campos dos Goytacazes - RJ'
df_bairro.head()

,Bairros,Endereco
0,Parque Jardim das Acácias,"Parque Jardim das Acácias, Campos dos Goytacaz..."
1,Parque Aeroporto,"Parque Aeroporto, Campos dos Goytacazes - RJ"
2,Parque Alberto Torres,"Parque Alberto Torres, Campos dos Goytacazes - RJ"
3,Parque Aldeia,"Parque Aldeia, Campos dos Goytacazes - RJ"
4,Alphaville,"Alphaville, Campos dos Goytacazes - RJ"


In [3]:
# Bases (facilidades)
file = '../data/enderecos.csv'
df_base = pd.read_csv(file,  encoding='utf-8', delimiter=';')
df_base

,Base,EnderecoBase
0,5 GBM,"Rua Rui Barbosa, 863 - Centro, Campos dos Goyt..."
1,DBM 1/5 GUARUS,"Rua Alci Ferreira - Codim, Campos dos Goytaca..."
2,HOSP. FERREIRA M,"Rua Rocha Leão, 02 - Centro, Campos dos Goytac..."
3,HOSP. G. GUARUS,"Avenida Senador José Carlos Pereira Pinto, 400..."
4,HOSP. SÃO JOSÉ,"Hospital São José, goytacazes, campos dos goyt..."
5,HOSP. TRAVESSÃO,"Avenida Antonio Luiz da Silveira - Centro, Cam..."
6,PS GUARUS,"Rua André Luis, 111 - Jardim Carioca, Campos ..."
7,SANDU,"Rua Saldanha Marinho, 59 - Centro, Campos dos ..."
8,UBS TOCOS,"Avenida José Pereira das Chagas, Tocos (Campos..."
9,UPA,"Avenida Carlos Alberto Chebabe, 517 - Parque G..."


## Geocode



**Obter coordenadas lat, long dos clientes e facilidades via API do google Maps**

In [4]:
# Obeter coodernadas geograficas das facilidades
if not pathlib.Path('../data/base.csv').exists():
    df_base['base_lat_long'] = df_base['EnderecoBase'].map(get_lat_log_from_addres)
    df_base.to_csv('../data/base.csv')
    
df_base = pd.read_csv('../data/base.csv', index_col=0)
df_base.head()

,Base,EnderecoBase,base_lat_long
0,5 GBM,"Rua Rui Barbosa, 863 - Centro, Campos dos Goyt...","(-21.7560766, -41.3163947)"
1,DBM 1/5 GUARUS,"Rua Alci Ferreira - Codim, Campos dos Goytaca...","(-21.709383, -41.2875301)"
2,HOSP. FERREIRA M,"Rua Rocha Leão, 02 - Centro, Campos dos Goytac...","(-21.7437715, -41.333381)"
3,HOSP. G. GUARUS,"Avenida Senador José Carlos Pereira Pinto, 400...","(-21.7413698, -41.3129263)"
4,HOSP. SÃO JOSÉ,"Hospital São José, goytacazes, campos dos goyt...","(-21.826222, -41.2879027)"


In [5]:
# Obeter cooderndas geograficas dos clientes
df_cliente = df_bairro.drop_duplicates(subset=['Endereco']).copy()

if not pathlib.Path('../data/clientes.csv').exists():
    df_cliente['cliente_lat_long'] = df_cliente['Endereco'].map(get_lat_log_from_addres)
    df_cliente.to_csv('../data/clientes.csv')
    
df_cliente = pd.read_csv('../data/clientes.csv', index_col=0)
df_cliente.head()


,Bairros,Endereco,cliente_lat_long
0,Parque Jardim das Acácias,"Parque Jardim das Acácias, Campos dos Goytacaz...","(-21.7546429, -41.3359471)"
1,Parque Aeroporto,"Parque Aeroporto, Campos dos Goytacazes - RJ","(-21.6969147, -41.307831)"
2,Parque Alberto Torres,"Parque Alberto Torres, Campos dos Goytacazes - RJ","(-21.7481364, -41.3415159)"
3,Parque Aldeia,"Parque Aldeia, Campos dos Goytacazes - RJ","(-21.7254055, -41.332284)"
4,Alphaville,"Alphaville, Campos dos Goytacazes - RJ","(-21.8143577, -41.2847415)"


**Gerar combinação das bases e clientes** 

In [8]:
def combine_source_destination(df_clients, df_facilities):
   ...
    
combin = []
for base in df_base.itertuples():
    combin.append(
        (
            base.Base,
            df_cliente['Bairros'].values,
            df_cliente['cliente_lat_long'].values,
            base.base_lat_long
         )
    )

In [9]:
df_coord = pd.DataFrame(
    combin, 
    columns=['Facilidade','Clientes', 'Coord_Clientes', 'Coord_Facilidade']
).explode(['Clientes',	'Coord_Clientes'],ignore_index=True)

df_coord.drop_duplicates(inplace=True)

In [10]:
df_coord['Coord_cliente_facilidade'] = df_coord[['Coord_Clientes', 'Coord_Facilidade']].apply(lambda x:(eval(x.iloc[0]),eval(x.iloc[1])), axis=1)
df_coord.head()

,Facilidade,Clientes,Coord_Clientes,Coord_Facilidade,Coord_cliente_facilidade
0,5 GBM,Parque Jardim das Acácias,"(-21.7546429, -41.3359471)","(-21.7560766, -41.3163947)","((-21.7546429, -41.3359471), (-21.7560766, -41..."
1,5 GBM,Parque Aeroporto,"(-21.6969147, -41.307831)","(-21.7560766, -41.3163947)","((-21.6969147, -41.307831), (-21.7560766, -41...."
2,5 GBM,Parque Alberto Torres,"(-21.7481364, -41.3415159)","(-21.7560766, -41.3163947)","((-21.7481364, -41.3415159), (-21.7560766, -41..."
3,5 GBM,Parque Aldeia,"(-21.7254055, -41.332284)","(-21.7560766, -41.3163947)","((-21.7254055, -41.332284), (-21.7560766, -41...."
4,5 GBM,Alphaville,"(-21.8143577, -41.2847415)","(-21.7560766, -41.3163947)","((-21.8143577, -41.2847415), (-21.7560766, -41..."


 ### Obter tempo base vs facilidades

In [11]:
if not pathlib.Path('../data/coordenadas.csv').exists():
    df_coord['Tempo'] = df_coord['Coord_cliente_facilidade'].map(lambda x: get_distance_matrix((x[0]),x[1]))
    df_coord.to_csv('../data/coordenadas.csv')

df_coord = pd.read_csv('../data/coordenadas.csv', index_col=0)

In [12]:
df_coord.head()

,Facilidade,Clientes,Coord_Clientes,Coord_Facilidade,Coord_cliente_facilidade,Tempo
0,5 GBM,Parque Jardim das Acácias,"(-21.7546429, -41.3359471)","(-21.7560766, -41.3163947)","((-21.7546429, -41.3359471), (-21.7560766, -41...",563
1,5 GBM,Parque Aeroporto,"(-21.6969147, -41.307831)","(-21.7560766, -41.3163947)","((-21.6969147, -41.307831), (-21.7560766, -41....",998
2,5 GBM,Parque Alberto Torres,"(-21.7481364, -41.3415159)","(-21.7560766, -41.3163947)","((-21.7481364, -41.3415159), (-21.7560766, -41...",674
3,5 GBM,Parque Aldeia,"(-21.7254055, -41.332284)","(-21.7560766, -41.3163947)","((-21.7254055, -41.332284), (-21.7560766, -41....",904
4,5 GBM,Alphaville,"(-21.8143577, -41.2847415)","(-21.7560766, -41.3163947)","((-21.8143577, -41.2847415), (-21.7560766, -41...",1132


**Gerar Matriz**

In [13]:
df_matrix = df_coord.pivot(columns='Facilidade',index='Clientes',  values='Tempo')
df_matrix.to_excel('../data/matriz_tempo.xlsx')

In [14]:
df_matrix

Facilidade,5 GBM,DBM 1/5 GUARUS,HOSP. FERREIRA M,HOSP. G. GUARUS,HOSP. SÃO JOSÉ,HOSP. TRAVESSÃO,PS GUARUS,SANDU,UBS Sartunino Braga,UBS TOCOS,UPA,UPH F. SÃO THOMÉ,UPH MORRO DO COCO,UPH ST EDUARDO
Clientes,,,,,,,,,,,,,,
Alphaville,1132,1997,1449,1345,527,2579,1450,972,1610,1348,1706,3454,4157,5609
Baixa Grande,3004,3869,3321,3217,1923,4451,3322,2844,655,2062,3578,1189,6029,7481
Boa Vista,3536,4401,3852,3749,2454,4982,3854,3375,1186,2432,4109,673,6561,8012
Caboio,2808,3673,3124,3021,1726,4254,3126,2647,1025,1404,3381,1925,5833,7284
Caju,508,768,150,591,1712,1331,368,811,2794,2422,412,4638,2909,4361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vila Industrial,982,223,909,800,2304,1050,826,1323,3386,3124,521,5231,2629,4080
Vila MANHÃES,1077,1942,1394,1290,974,2524,1455,999,1524,1571,1650,3369,4102,5554
Vila Nova de Campos,2751,2197,2625,2570,4073,1425,2596,3093,5155,4893,2238,7000,1052,2580


## Adequar dados para Sistema de Informação Geográfica 

In [38]:
def extract_lat_long(df, lat_long, path=None):
    df['lat'] = df[lat_long].map(lambda x: x[1:-1].split(',')[0])
    df['long'] = df[lat_long].map(lambda x: x[1:-1].split(',')[1])
    if path:
        df.to_csv(path)
    return df
    
extract_lat_long(
    df_base,
    'base_lat_long',
    r'C:\Users\fabio\OneDrive\Orientacao\Leonardo\Mapas\dados_geograficos\base.csv')

,Base,EnderecoBase,base_lat_long,lat,long
0,5 GBM,"Rua Rui Barbosa, 863 - Centro, Campos dos Goyt...","(-21.7560766, -41.3163947)",-21.7560766,-41.3163947
1,DBM 1/5 GUARUS,"Rua Alci Ferreira - Codim, Campos dos Goytaca...","(-21.709383, -41.2875301)",-21.709383,-41.2875301
2,HOSP. FERREIRA M,"Rua Rocha Leão, 02 - Centro, Campos dos Goytac...","(-21.7437715, -41.333381)",-21.7437715,-41.333381
3,HOSP. G. GUARUS,"Avenida Senador José Carlos Pereira Pinto, 400...","(-21.7413698, -41.3129263)",-21.7413698,-41.3129263
4,HOSP. SÃO JOSÉ,"Hospital São José, goytacazes, campos dos goyt...","(-21.826222, -41.2879027)",-21.826222,-41.2879027
5,HOSP. TRAVESSÃO,"Avenida Antonio Luiz da Silveira - Centro, Cam...","(-21.6093672, -41.3123019)",-21.6093672,-41.3123019
6,PS GUARUS,"Rua André Luis, 111 - Jardim Carioca, Campos ...","(-21.7454022, -41.3228796)",-21.7454022,-41.3228796
7,SANDU,"Rua Saldanha Marinho, 59 - Centro, Campos dos ...","(-21.7642258, -41.3148822)",-21.7642258,-41.3148822
8,UBS TOCOS,"Avenida José Pereira das Chagas, Tocos (Campos...","(-21.9007729, -41.2831951)",-21.9007729,-41.2831951
9,UPA,"Avenida Carlos Alberto Chebabe, 517 - Parque G...","(-21.7258082, -41.3210406)",-21.7258082,-41.3210406


In [39]:
extract_lat_long(
    df_cliente,
    'cliente_lat_long', 
    r'C:\Users\fabio\OneDrive\Orientacao\Leonardo\Mapas\dados_geograficos\bairros.csv'
)

,Bairros,Endereco,cliente_lat_long,lat,long
0,Parque Jardim das Acácias,"Parque Jardim das Acácias, Campos dos Goytacaz...","(-21.7546429, -41.3359471)",-21.7546429,-41.3359471
1,Parque Aeroporto,"Parque Aeroporto, Campos dos Goytacazes - RJ","(-21.6969147, -41.307831)",-21.6969147,-41.307831
2,Parque Alberto Torres,"Parque Alberto Torres, Campos dos Goytacazes - RJ","(-21.7481364, -41.3415159)",-21.7481364,-41.3415159
3,Parque Aldeia,"Parque Aldeia, Campos dos Goytacazes - RJ","(-21.7254055, -41.332284)",-21.7254055,-41.332284
4,Alphaville,"Alphaville, Campos dos Goytacazes - RJ","(-21.8143577, -41.2847415)",-21.8143577,-41.2847415
...,...,...,...,...,...
160,Pracinha I da Vila Romana,"Pracinha I da Vila Romana, Campos dos Goytacaz...","(-21.8373031, -41.2568248)",-21.8373031,-41.2568248
161,Parque Visc. de Ururai,"Parque Visc. de Ururai, Campos dos Goytacazes ...","(-21.7208807, -41.3291496)",-21.7208807,-41.3291496
162,Parque Vista Alegre,"Parque Vista Alegre, Campos dos Goytacazes - RJ","(-21.7755557, -41.3009324)",-21.7755557,-41.3009324
163,Vivendas Do Coqueiro,"Vivendas Do Coqueiro, Campos dos Goytacazes - RJ","(-21.7920989, -41.2954365)",-21.7920989,-41.2954365
